In [67]:
import math
import copy
import random
import numpy as np

# Data Setup
tam = {
    'Q1’26': 21.8, 'Q2’26': 27.4, 'Q3’26': 34.9, 'Q4’26': 39.0,
    'Q1’27': 44.7, 'Q2’27': 51.5, 'Q3’27': 52.5, 'Q4’27': 53.5
}

node_data = {
    'Node1': {
        'GB_per_wafer': {q: 100000 for q in tam},
        'Yield': {q: 0.98 for q in tam},
    },
    'Node2': {
        'GB_per_wafer': {q: 150000 for q in tam},
        'Yield': {
            'Q1’26': 0.60, 'Q2’26': 0.82, 'Q3’26': 0.95, 'Q4’26': 0.98,
            'Q1’27': 0.98, 'Q2’27': 0.98, 'Q3’27': 0.98, 'Q4’27': 0.98
        },
    },
    'Node3': {
        'GB_per_wafer': {q: 270000 for q in tam},
        'Yield': {
            'Q1’26': 0.20, 'Q2’26': 0.25, 'Q3’26': 0.35, 'Q4’26': 0.50,
            'Q1’27': 0.65, 'Q2’27': 0.85, 'Q3’27': 0.95, 'Q4’27': 0.98
        },
    }
}

workstations = {
    'A': {'initial': 10, 'util': 0.78, 'capex': 3.0e6, 'nodes': {'Node1': 4.0, 'Node2': 4.0, 'Node3': 4.0}},
    'B': {'initial': 18, 'util': 0.76, 'capex': 6.0e6, 'nodes': {'Node1': 6.0, 'Node2': 9.0, 'Node3': 15.0}},
    'C': {'initial': 5, 'util': 0.80, 'capex': 2.2e6, 'nodes': {'Node1': 2.0, 'Node2': 2.0, 'Node3': 5.4}},
    'D': {'initial': 11, 'util': 0.80, 'capex': 3.0e6, 'nodes': {'Node1': 5.0, 'Node2': 5.0, 'Node3': 0.0}},
    'E': {'initial': 15, 'util': 0.76, 'capex': 3.5e6, 'nodes': {'Node1': 5.0, 'Node2': 10.0, 'Node3': 0.0}},
    'F': {'initial': 2, 'util': 0.80, 'capex': 6.0e6, 'nodes': {'Node1': 0.0, 'Node2': 1.8, 'Node3': 5.8}},
    'G': {'initial': 23, 'util': 0.70, 'capex': 2.1e6, 'nodes': {'Node1': 12.0, 'Node2': 0.0, 'Node3': 16.0}},
    'H': {'initial': 3, 'util': 0.85, 'capex': 1.8e6, 'nodes': {'Node1': 2.1, 'Node2': 0.0, 'Node3': 0.0}},
    'I': {'initial': 4, 'util': 0.75, 'capex': 3.0e6, 'nodes': {'Node1': 0.0, 'Node2': 6.0, 'Node3': 0.0}},
    'J': {'initial': 1, 'util': 0.60, 'capex': 8.0e6, 'nodes': {'Node1': 0.0, 'Node2': 0.0, 'Node3': 2.1}},
}

# initial_loading = {
#     'Node1': [12000, 14500, 16320, 16320, 16320, 16320, 16316, 16319],
#     'Node2': [5000, 3024, 4032, 4032, 4032, 4032, 3112, 4015],
#     'Node3': [1000, 2780, 4171, 5561, 6951, 8342, 8327, 7860],
# }

# Checkpoint at iteration 781000: Best Profit = $447.08M
initial_loading = {
    'Node1': [12000, 12006, 9640, 8178, 6981, 6981, 6878, 6860] ,
    'Node2': [5000, 6900, 8969, 9486, 10080, 10080, 9917, 10078] ,
    'Node3': [1000, 3500, 6000, 7096, 7994, 7994, 8033, 7995]
}

quarters = list(tam.keys())
loading = {q: {'Node1': initial_loading['Node1'][i], 
            'Node2': initial_loading['Node2'][i], 
            'Node3': initial_loading['Node3'][i]} 
          for i, q in enumerate(quarters)}

In [68]:
# Profit Calculation Function
def calculate_profit(loading):
    total_gb, capex = 0, 0
    tool_counts = {ws: workstations[ws]['initial'] for ws in workstations}
    
    for q in quarters:
        # Compute GB production for the quarter and check TAM constraints
        gb = sum(13 * loading[q][node] *
                 node_data[node]['GB_per_wafer'][q] *
                 node_data[node]['Yield'][q]
                 for node in ['Node1', 'Node2', 'Node3'])
        if not (tam[q] - 2 <= gb / 1e9 <= tam[q] + 2):
            return -float('inf')
        total_gb += gb
        
        # Calculate tool requirements and CAPEX
        for ws in workstations:
            load = sum(loading[q][node] * workstations[ws]['nodes'].get(node, 0)
                       for node in ['Node1', 'Node2', 'Node3'])
            req = math.ceil(load / (7 * 24 * 60 * workstations[ws]['util']))
            if req > tool_counts[ws]:
                capex += (req - tool_counts[ws]) * workstations[ws]['capex']
                tool_counts[ws] = req
    
    return total_gb * 0.002 - capex

In [69]:
# Extended evaluation function that also returns workstation counts per quarter.
def evaluate_solution(sol):
    total_gb, capex = 0, 0
    tool_counts = {ws: workstations[ws]['initial'] for ws in workstations}
    tool_counts_by_quarter = {}
    
    for q in quarters:
        gb = sum(13 * sol[q][node] *
                 node_data[node]['GB_per_wafer'][q] *
                 node_data[node]['Yield'][q]
                 for node in ['Node1', 'Node2', 'Node3'])
        if not (tam[q] - 2 <= gb / 1e9 <= tam[q] + 2):
            return -float('inf'), {}
        total_gb += gb
        for ws in workstations:
            load_val = sum(sol[q][node] * workstations[ws]['nodes'].get(node, 0)
                           for node in ['Node1', 'Node2', 'Node3'])
            req = math.ceil(load_val / (7 * 24 * 60 * workstations[ws]['util']))
            if req > tool_counts[ws]:
                capex += (req - tool_counts[ws]) * workstations[ws]['capex']
                tool_counts[ws] = req
        tool_counts_by_quarter[q] = copy.deepcopy(tool_counts)
    profit = total_gb * 0.002 - capex
    return profit, tool_counts_by_quarter

In [70]:
# Local perturbation function (generates a neighboring solution)
def perturb(sol):
    new_sol = copy.deepcopy(sol)
    # Allow perturbations on all quarters except the first
    q = random.choice(quarters[1:])
    node = random.choice(['Node1', 'Node2', 'Node3'])
    delta = random.randint(-2500, 2500)
    new_val = new_sol[q][node] + delta
    
    if new_val < 0:
        return new_sol
    
    idx = quarters.index(q)
    if idx > 0 and abs(new_val - new_sol[quarters[idx-1]][node]) > 2500:
        return new_sol
    if idx < len(quarters) - 1 and abs(new_val - new_sol[quarters[idx+1]][node]) > 2500:
        return new_sol
    
    new_sol[q][node] = new_val
    return new_sol

In [71]:
# Function to generate a random solution based on the initial loading
def generate_random_solution():
    sol = copy.deepcopy(loading)
    # Apply a few random perturbations to introduce diversity
    for _ in range(random.randint(1, 3)):
        sol = perturb(sol)
    return sol

In [72]:
# Parameters for ABC Algorithm
NP = 10         # Number of food sources (solutions)
limit = 50      # Maximum trial counter before abandonment
max_iter = 1000000 # Maximum number of iterations
checkpoint_interval = 1000 # Interval to print best solution

# Initialize population (food sources), profit values, and trial counters
food_sources = [generate_random_solution() for _ in range(NP)]
profits = [calculate_profit(sol) for sol in food_sources]
trials = [0 for _ in range(NP)]

# Track the best solution found so far
best_index = np.argmax(profits)
best_profit = profits[best_index]
best_solution = copy.deepcopy(food_sources[best_index])


In [73]:
# Main loop of the Artificial Bee Colony algorithm
for it in range(max_iter):
    # --- Employed Bee Phase ---
    for i in range(NP):
        candidate = perturb(food_sources[i])
        candidate_profit = calculate_profit(candidate)
        if candidate_profit > profits[i]:
            food_sources[i] = candidate
            profits[i] = candidate_profit
            trials[i] = 0
        else:
            trials[i] += 1

    # --- Calculate Probabilities for Onlooker Bees ---
    fitness = [p if p > 0 else 0 for p in profits]
    total_fitness = sum(fitness)
    if total_fitness == 0:
        probabilities = [1 / NP] * NP
    else:
        probabilities = [f / total_fitness for f in fitness]
    
    # --- Onlooker Bee Phase ---
    onlooker_count = 0
    i = 0
    while onlooker_count < NP:
        if random.random() < probabilities[i]:
            candidate = perturb(food_sources[i])
            candidate_profit = calculate_profit(candidate)
            if candidate_profit > profits[i]:
                food_sources[i] = candidate
                profits[i] = candidate_profit
                trials[i] = 0
            else:
                trials[i] += 1
            onlooker_count += 1
        i = (i + 1) % NP

    # --- Scout Bee Phase ---
    for i in range(NP):
        if trials[i] > limit:
            food_sources[i] = generate_random_solution()
            profits[i] = calculate_profit(food_sources[i])
            trials[i] = 0

    # Update best solution if a better one is found
    current_best_index = np.argmax(profits)
    if profits[current_best_index] > best_profit:
        best_profit = profits[current_best_index]
        best_solution = copy.deepcopy(food_sources[current_best_index])

    # Print checkpoint at defined intervals
    if (it + 1) % checkpoint_interval == 0:
        current_profit, current_tool_counts = evaluate_solution(best_solution)
        # Reformat best_solution into node-centric dictionary
        optimized_loading = {'Node1': [], 'Node2': [], 'Node3': []}
        for q in quarters:
            optimized_loading['Node1'].append(best_solution[q]['Node1'])
            optimized_loading['Node2'].append(best_solution[q]['Node2'])
            optimized_loading['Node3'].append(best_solution[q]['Node3'])
        print(f"\nCheckpoint at iteration {it + 1}: Best Profit = ${best_profit/1e6:.2f}M")
        print("Optimized Loading:")
        print("optimized_loading = {")
        print("    'Node1':", optimized_loading['Node1'], ",")
        print("    'Node2':", optimized_loading['Node2'], ",")
        print("    'Node3':", optimized_loading['Node3'])
        print("}")
        print("\nWorkstation counts by quarter:")
        for q in quarters:
            print(f"{q}: {current_tool_counts[q]}")

# Final Output
print(f"Optimized Profit: ${best_profit/1e6:.2f}M")
print("Optimized Loading:")

# Final Output: Compute final loading and workstation counts
final_profit, final_tool_counts = evaluate_solution(best_solution)
optimized_loading = {'Node1': [], 'Node2': [], 'Node3': []}
for q in quarters:
    optimized_loading['Node1'].append(best_solution[q]['Node1'])
    optimized_loading['Node2'].append(best_solution[q]['Node2'])
    optimized_loading['Node3'].append(best_solution[q]['Node3'])

print("\nFinal optimized_loading = {")
print("    'Node1':", optimized_loading['Node1'], ",")
print("    'Node2':", optimized_loading['Node2'], ",")
print("    'Node3':", optimized_loading['Node3'])
print("}")
print("\nFinal Workstation counts by quarter:")
for q in quarters:
    print(f"{q}: {final_tool_counts[q]}")


Checkpoint at iteration 1000: Best Profit = $471.62M
Optimized Loading:
optimized_loading = {
    'Node1': [12000, 12006, 9640, 8178, 6981, 6981, 6878, 6860] ,
    'Node2': [5000, 6900, 8969, 9486, 10080, 10080, 9917, 10078] ,
    'Node3': [1000, 3500, 6000, 7096, 7994, 7994, 8033, 7995]
}

Workstation counts by quarter:
Q1’26: {'A': 10, 'B': 18, 'C': 5, 'D': 11, 'E': 15, 'F': 2, 'G': 23, 'H': 3, 'I': 4, 'J': 1}
Q2’26: {'A': 12, 'B': 25, 'C': 8, 'D': 12, 'E': 17, 'F': 5, 'G': 29, 'H': 3, 'I': 6, 'J': 2}
Q3’26: {'A': 13, 'B': 30, 'C': 9, 'D': 12, 'E': 18, 'F': 7, 'G': 30, 'H': 3, 'I': 8, 'J': 3}
Q4’26: {'A': 13, 'B': 32, 'C': 10, 'D': 12, 'E': 18, 'F': 8, 'G': 30, 'H': 3, 'I': 8, 'J': 3}
Q1’27: {'A': 13, 'B': 33, 'C': 10, 'D': 12, 'E': 18, 'F': 8, 'G': 30, 'H': 3, 'I': 8, 'J': 3}
Q2’27: {'A': 13, 'B': 33, 'C': 10, 'D': 12, 'E': 18, 'F': 8, 'G': 30, 'H': 3, 'I': 8, 'J': 3}
Q3’27: {'A': 13, 'B': 33, 'C': 10, 'D': 12, 'E': 18, 'F': 8, 'G': 30, 'H': 3, 'I': 8, 'J': 3}
Q4’27: {'A': 13, 'B':